In [14]:
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter, ParameterVector
from qiskit.circuit.library import ZFeatureMap
from qiskit.visualization import plot_histogram, plot_distribution

from qiskit_machine_learning.neural_networks import SamplerQNN
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier
from qiskit_algorithms.optimizers import COBYLA
from qiskit.primitives import Sampler

from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt

In [15]:
# Method to construct the Convolutional Layer for the QCNN
def conv_layer(qc:QuantumCircuit(), para:ParameterVector, iter:int) -> QuantumCircuit():
    start = 0 if qc.num_qubits == 2 else -1
    iter = 3 if iter == 0 else iter
    for i in range(start, qc.num_qubits-1):
        qc.rz(-np.pi / 2, i+1)
        qc.cx(i+1, i)
        qc.rz(para[(3*i)+iter], i)
        qc.ry(para[(3*i)+1+iter], i+1)
        qc.cx(i, i+1)
        qc.ry(para[(3*i)+2+iter], i+1)
        qc.cx(i+1, i)
        qc.rz(np.pi / 2, i)
        qc.barrier()
    if iter == 3:
        iter = 0
    if qc.num_qubits == 2:
        iter += 3
    else: 
        iter += (3*(qc.num_qubits)) 
    return qc, iter

# Method to construct the Pooling Layer for the QCNN
def pool_layer(qc:QuantumCircuit(), para:ParameterVector, iter:int) -> QuantumCircuit():
    half = int(qc.num_qubits/2)
    for i in range(half):
        qc.rz(-np.pi / 2, i+half)
        qc.cx(i+half, i)
        qc.rz(para[(3*i)+iter], i)
        qc.ry(para[(3*i)+1+iter], i+half)
        qc.cx(i, i+half)
        qc.ry(para[(3*i)+2+iter], i+half)
        qc.barrier()
    iter += (half*3)
    return qc, iter

# Method that builds the entire circuit and layers systematically
def qcnn_circuit(num_qubits:int, num_outputs:int) -> QuantumCircuit():
    qc = QuantumCircuit(num_qubits)
    num_outputs *= 2
    num_layers = np.log2(num_qubits) # Calculates number of convolutional/pooling layers

    feature_map = ZFeatureMap(num_qubits)  # Pulls in Qiskit's built in ZZFeatureMap
    qc.compose(feature_map, range(num_qubits), inplace=True) # Adds ZFeatureMap to circuit for initialization
    
    parameter_vec = ParameterVector(name="θ", length=int(9*(2**num_layers))-12) # Calculates number of parameters needed
   
    iter = 0 # Tracks where in the Parameter Vector to put the next parameter
    index = num_qubits # Divides itself in half at each layer
    layer = 1 # Tracks the current layer being constructed
    while index > int(num_outputs): # Adds 1 convolution layer and 1 pooling layer then iterates until the desired amount of outputs remain
        conv_qc = QuantumCircuit(index)
        pool_qc = QuantumCircuit(index)
        
        # Adds 1 convolutional layer to the circuit by converting the convolutional circuit to a gate
        conv_qc, iter = conv_layer(conv_qc, parameter_vec, iter)
        qc.compose(conv_qc.to_instruction(label=f'Convolution Layer {layer}'), range(int(num_qubits - index), num_qubits), inplace=True)
        
        # Adds 1 pooling layer to the circuit by converting the pooling circuit to a gate
        pool_qc, iter = pool_layer(pool_qc, parameter_vec, iter)
        qc.compose(pool_qc.to_instruction(label=f'Pooling Layer {layer}'), range(int(num_qubits - index), num_qubits), inplace=True)

        # Handle tracker increment or decrement
        index = int(index / 2)
        layer += 1

    # Constructs weight parameter by removing ZFeatureMap's parameters from the total
    weight_params = qc.parameters - feature_map.parameters
    return qc, feature_map.parameters, weight_params

# Interpreting function for the classifier
def func(x):
    n = 2 # n must match out_qubits in cell below
    return x % 2**n 

In [16]:
# Loading data
data = np.genfromtxt("seeds_dataset.txt", dtype=str)

# Splitting the data to features and labels
features = data[:, :-1].astype(float)  
labels = data[:, -1]

features = np.hstack((features, np.zeros((features.shape[0], 1))))
features_list = [np.array(row) for row in features]

# Split data into test and training sets
trainer, tester, trainer_ans, tester_ans = train_test_split(features_list, labels, test_size=0.25, random_state=246)

x = np.asarray(trainer)
y = np.asarray(trainer_ans)

In [17]:
# Declares the Quantum Convolutional Circuit and extracts the input and weigh parameters from the qcnn_circuit method
out_qubits = 2 # If using more than 4 output types adjust '2' to 2^n types
qcnn, input_params, weight_params = qcnn_circuit(len(features[0]), out_qubits) 

# Constructs the neural network and parameters for the classifier
qcnn_sam = SamplerQNN(circuit=qcnn.decompose(), input_params=input_params, weight_params=weight_params,
                       interpret=func, output_shape=(2**out_qubits))

# Declares QCNN Classifier
classifier = NeuralNetworkClassifier(qcnn_sam, optimizer=COBYLA(maxiter=100))

In [20]:
########### UNCOMMENT ONLY THE OPTION YOU ARE TRYING TO DO! ###########

# Uncomment this section to create and save a new model
'''
classifier.fit(x, y)
classifier.save("seed_classifier.model")
'''
 
# Uncomment this section to load or warm start a model
'''
classifier = NeuralNetworkClassifier.load("seed_classifier_8m1000.model")
'''

'\nclassifier = NeuralNetworkClassifier.load("seed_classifier_8m1000.model")\n'

In [19]:
y_predict = classifier.predict(tester)
x = np.asarray(tester)
y = np.asarray(tester_ans)

print(f"Accuracy from the test data : {np.round(100 * classifier.score(x, y), 2)}%")

Accuracy from the test data : 58.49%
